# **Global Solution 2 - Motor de Orientação de Habilidades (MOH)**

**Dinamic Programming**

---

### **Integrantes:**

* **Camila Pedroza da Cunha** – RM 558768
* **Renan Dias Utida** - RM 558540

**Sala:** 2ESPW

**Data de Entrega:** 21/11/2025

---

# **Introdução**

## **📊 Contexto do Problema**

O **Motor de Orientação de Habilidades (MOH)** é um sistema inteligente projetado para guiar profissionais na aquisição estratégica de habilidades para maximizar o valor de carreira e adaptabilidade em um mercado em constante transformação.

### **🎯 Objetivos Principais**

O sistema deve:
- **Otimizar** a sequência de aquisição de habilidades sob múltiplas restrições
- **Considerar** pré-requisitos obrigatórios entre habilidades
- **Incorporar** incerteza nos valores de mercado das habilidades
- **Validar** a integridade do grafo de dependências (ciclos e órfãos)
- **Recomendar** próximas habilidades baseado em cenários futuros

### **📐 Formulação Matemática**

**Variáveis de Decisão:**
- $x_i \in \{0,1\}$: indica se habilidade $i$ foi selecionada

**Função Objetivo:**
$$\max \sum_{i=1}^{n} V_i \cdot x_i$$

**Restrições:**
1. Tempo total: $\sum_{i=1}^{n} T_i \cdot x_i \leq 350$ horas
2. Complexidade cumulativa: $\sum_{i=1}^{n} C_i \cdot x_i \leq 30$
3. Pré-requisitos: $x_j = 1 \implies x_i = 1, \forall i \in PreReq(j)$
4. Objetivo: $x_{S6} = 1$ (alcançar IA Generativa Ética)

### **🔢 Dataset Base**

O sistema trabalha com **12 habilidades** organizadas em:
- **5 Habilidades Básicas** (sem pré-requisitos): S1, S2, S7, H10, H12
- **5 Habilidades Críticas**: S3, S5, S7, S8, S9
- **1 Habilidade Objetivo**: S6 (IA Generativa Ética)
- **Habilidades Avançadas**: S4, H11

### **🚀 Desafios a Resolver**

1. **Desafio 1** - Caminho de Valor Máximo (DP + Monte Carlo)
2. **Desafio 2** - Verificação Crítica (120 Permutações)
3. **Desafio 3** - Pivô Mais Rápido (Guloso vs Ótimo)
4. **Desafio 4** - Trilhas Paralelas (Merge/Quick Sort)
5. **Desafio 5** - Recomendação com Horizonte (DP Look-ahead)

---

# **Configuração Inicial e Imports**

## **📦 Bibliotecas e Módulos**

Carregamento de todas as bibliotecas necessárias e dos módulos customizados desenvolvidos para este projeto.

In [2]:
# ============================================================================
# IMPORTS PADRÃO
# ============================================================================

import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from typing import Dict, List, Any, Tuple

# Suprime warnings
warnings.filterwarnings('ignore')

# ============================================================================
# IMPORTS DOS MÓDULOS CUSTOMIZADOS
# ============================================================================

# Configurações e constantes
from src.config import (
    MAX_TIME_HOURS,
    MAX_COMPLEXITY,
    TARGET_SKILL,
    CRITICAL_SKILLS,
    BASIC_SKILLS,
    N_MONTE_CARLO_SCENARIOS,
    GLOBAL_SEED,
    DATA_DIR,
    OUTPUTS_DIR,
    SKILLS_DATASET_FILE,
    print_header,
    format_hours,
    format_path,
    format_constraint
)

# Estruturas de dados (Grafo)
from src.graph_structures import (
    SkillGraph,
    load_skills_from_json,
    build_graph_from_dict,
    build_graph_from_file,
    print_graph_summary
)

# Validação do grafo (CRÍTICO)
from src.graph_validation import (
    validate_graph,
    detect_cycles,
    detect_orphan_nodes,
    print_validation_report,
    ensure_valid_graph
)

# Desafio 1: DP Knapsack + Monte Carlo
from src.desafio1_dp_knapsack import (
    dp_knapsack_2d,
    solve_desafio1_complete,
    KnapsackSolution
)

# Monte Carlo
from src.monte_carlo import (
    generate_scenarios,
    run_monte_carlo,
    compare_deterministic_vs_stochastic,
    MonteCarloResult
)

# Visualizações
from src.visualization import (
    plot_monte_carlo_distribution,
    plot_permutations_comparison,
    plot_greedy_vs_optimal,
    create_dashboard
)

# Análises estatísticas
from src.analysis import (
    calculate_summary_statistics,
    compare_algorithms,
    generate_final_report
)

# Decorators de performance
from src.decorators import measure_performance

print("=" * 70)
print("✅ TODOS OS MÓDULOS IMPORTADOS COM SUCESSO")
print("=" * 70)

AttributeError: module 'src.config' has no attribute 'WELCOME_MESSAGE'

# 🎯 **Solução dos Problemas**

## ⚙️ **Configuração Inicial e Imports**

Nesta seção, importamos todos os módulos necessários para executar os 5 desafios.

### 🔧 **Setup e Instalação**

**Objetivo:** Garantir que todas as dependências estejam instaladas e importadas

**Bibliotecas necessárias:**
- `numpy` - Computação numérica
- `matplotlib` - Visualizações
- `seaborn` - Gráficos estatísticos
- `scipy` - Análises estatísticas avançadas (opcional)

In [38]:
# Instalação de dependências (caso necessário)
import subprocess
import sys

def install_package(package):
    """Instala pacote se não estiver disponível."""
    try:
        __import__(package)
        print(f"✅ {package} já instalado")
    except ImportError:
        print(f"⚙️  Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        print(f"✅ {package} instalado com sucesso")

# Lista de dependências
dependencies = ['numpy', 'matplotlib', 'seaborn']

print("🔧 Verificando dependências...\n")
for dep in dependencies:
    install_package(dep)

# scipy é opcional (pode falhar em alguns ambientes)
try:
    install_package('scipy')
except:
    print("⚠️  scipy não instalado (análises estatísticas avançadas não disponíveis)")

print("\n✅ Setup concluído!")

🔧 Verificando dependências...

✅ numpy já instalado
✅ matplotlib já instalado
✅ seaborn já instalado
✅ scipy já instalado

✅ Setup concluído!


In [54]:
# ============================================================================
# IMPORTS BÁSICOS
# ============================================================================

import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Suprime warnings
warnings.filterwarnings('ignore')

# Banner inicial
print("=" * 80)
print("MOTOR DE ORIENTAÇÃO DE HABILIDADES (MOH)")
print("Global Solution - Dynamic Programming")
print("=" * 80)
print("✅ Imports básicos carregados com sucesso!")


MOTOR DE ORIENTAÇÃO DE HABILIDADES (MOH)
Global Solution - Dynamic Programming
✅ Imports básicos carregados com sucesso!


In [57]:
# ============================================================================
# IMPORTS DO PROJETO - CONFIGURAÇÃO
# ============================================================================

from src.config import (
    MAX_TIME_HOURS, MAX_COMPLEXITY,
    TARGET_SKILL, CRITICAL_SKILLS, BASIC_SKILLS,
    MIN_ADAPTABILITY_TARGET,
    RECOMMENDATION_HORIZON_YEARS, N_RECOMMENDATIONS,
    MARKET_SCENARIOS,
    GLOBAL_SEED,
)

print("✅ Configurações carregadas!")
print(f"   • Tempo máximo: {MAX_TIME_HOURS}h")
print(f"   • Complexidade máxima: {MAX_COMPLEXITY}")
print(f"   • Objetivo: {TARGET_SKILL}")
print(f"   • Seed global: {GLOBAL_SEED}")

✅ Configurações carregadas!
   • Tempo máximo: 350h
   • Complexidade máxima: 30
   • Objetivo: S6
   • Seed global: 42


In [58]:
# ============================================================================
# IMPORTS DO PROJETO - GRAFOS
# ============================================================================

from src.graph_structures import (
    SkillGraph,
    load_skills_from_json,
    build_graph_from_dict,
    build_graph_from_file,
    print_graph_summary
)

from src.graph_validation import (
    validate_graph,
    print_validation_report,
    ensure_valid_graph
)

print("✅ Módulos de grafo carregados!")

✅ Módulos de grafo carregados!


In [59]:
# ============================================================================
# IMPORTS DO PROJETO - DESAFIOS
# ============================================================================

# Desafio 1
from src.desafio1_dp_knapsack import (
    solve_complete as solve_desafio1,
    print_solution_details,
    save_desafio1_results
)

from src.monte_carlo import (
    MonteCarloResult,
    generate_scenarios,
    run_monte_carlo,
    print_monte_carlo_summary
)

# Desafio 2
from src.desafio2_permutations import (
    solve_complete as solve_desafio2,
    print_order_details,
    save_desafio2_results
)

# Desafio 3
from src.desafio3_greedy import (
    solve_complete as solve_desafio3,
    save_desafio3_results
)

# Desafio 4
from src.desafio4_sorting import (
    solve_complete as solve_desafio4,
    save_desafio4_results
)

# Desafio 5
from src.desafio5_recommendation import (
    solve_complete as solve_desafio5,
    print_recommendation_details,
    save_desafio5_results
)

print("✅ Todos os desafios carregados!")

✅ Todos os desafios carregados!


In [60]:
# ============================================================================
# IMPORTS DO PROJETO - ANÁLISE E VISUALIZAÇÃO
# ============================================================================

from src.analysis import (
    compare_deterministic_vs_stochastic,
    analyze_permutations_costs,
    compare_greedy_vs_optimal,
    compare_all_algorithms,
    calculate_aggregate_metrics,
    generate_summary_report
)

from src.visualization import (
    plot_monte_carlo_distribution,
    plot_permutations_comparison,
    plot_greedy_vs_optimal,
    plot_market_scenarios,
    plot_algorithms_complexity_comparison,
    create_dashboard,
    save_all_plots
)

print("✅ Módulos de análise e visualização carregados!")

✅ Módulos de análise e visualização carregados!


In [61]:
# ============================================================================
# CONFIGURAÇÃO DE VISUALIZAÇÕES
# ============================================================================

# Estilo dos gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

# Diretórios do projeto
from src.config import DATA_DIR, OUTPUTS_DIR

# Cria diretório de outputs/figures se não existir
(OUTPUTS_DIR / 'figures').mkdir(exist_ok=True, parents=True)

print("✅ Visualizações configuradas!")
print(f"   • Diretório de dados: {DATA_DIR}")
print(f"   • Diretório de outputs: {OUTPUTS_DIR}")

✅ Visualizações configuradas!
   • Diretório de dados: c:\Users\ronal\Downloads\gs-dynamic-programming\data
   • Diretório de outputs: c:\Users\ronal\Downloads\gs-dynamic-programming\outputs


## 🔍 **Carregamento e Validação do Grafo de Habilidades**

Antes de executar QUALQUER otimização, é **OBRIGATÓRIO** validar o grafo para:
- ✅ Detectar **ciclos** (dependências circulares)
- ✅ Detectar **nós órfãos** (pré-requisitos inexistentes)

Esta validação é **CRÍTICA** e essencial antes de começarmos os nossas resoluções dos desafios

### 📊 **Carregando Dataset**

In [62]:
# ============================================================================
# CARREGAMENTO DO DATASET
# ============================================================================

from src.config import SKILLS_DATASET_FILE

print("=" * 70)
print("CARREGAMENTO DO DATASET DE HABILIDADES")
print("=" * 70)

# Carrega habilidades do JSON
skills_data = load_skills_from_json(SKILLS_DATASET_FILE)

print(f"\n✅ Dataset carregado com sucesso!")
print(f"   • Arquivo: {SKILLS_DATASET_FILE}")
print(f"   • Total de habilidades: {len(skills_data)}")

# Mostra preview das habilidades
print(f"\n📋 Habilidades carregadas:")
for skill_id, metadata in skills_data.items():
    prereqs = metadata.get('pre_requisitos', [])
    prereqs_str = ', '.join(prereqs) if prereqs else 'Nenhum'
    print(f"   • {skill_id}: {metadata['nome']}")
    print(f"     T={metadata['tempo_horas']}h, V={metadata['valor']}, C={metadata['complexidade']}, Pré-reqs=[{prereqs_str}]")

CARREGAMENTO DO DATASET DE HABILIDADES

✅ Dataset carregado com sucesso!
   • Arquivo: c:\Users\ronal\Downloads\gs-dynamic-programming\data\skills_dataset.json
   • Total de habilidades: 12

📋 Habilidades carregadas:
   • S1: Programação Básica (Python)
     T=80h, V=3, C=4, Pré-reqs=[Nenhum]
   • S2: Modelagem de Dados (SQL)
     T=60h, V=4, C=3, Pré-reqs=[Nenhum]
   • S3: Algoritmos Avançados
     T=100h, V=7, C=8, Pré-reqs=[S1]
   • S4: Fundamentos de Machine Learning
     T=120h, V=8, C=9, Pré-reqs=[S1, S3]
   • S5: Visualização de Dados (BI)
     T=40h, V=6, C=5, Pré-reqs=[S2]
   • S6: IA Generativa Ética
     T=150h, V=10, C=10, Pré-reqs=[S4]
   • S7: Estruturas em Nuvem (AWS/Azure)
     T=70h, V=5, C=7, Pré-reqs=[Nenhum]
   • S8: APIs e Microsserviços
     T=90h, V=6, C=6, Pré-reqs=[S1]
   • S9: DevOps & CI/CD
     T=110h, V=9, C=8, Pré-reqs=[S7, S8]
   • H10: Segurança de Dados
     T=60h, V=5, C=6, Pré-reqs=[Nenhum]
   • H11: Análise de Big Data
     T=90h, V=8, C=8, Pré-reqs=

### 🏗️ **Construindo o Grafo**

In [63]:
# ============================================================================
# CONSTRUÇÃO DO GRAFO
# ============================================================================

print("\n" + "=" * 70)
print("CONSTRUÇÃO DO GRAFO DE HABILIDADES")
print("=" * 70)

# Constrói o grafo
graph = build_graph_from_dict(skills_data)

print(f"\n✅ Grafo construído com sucesso!")
print(f"   • Total de nós: {len(graph)}")
print(f"   • Total de arestas (pré-requisitos): {sum(len(graph.adjacency_list[node]) for node in graph.nodes)}")

# Habilidades básicas (sem pré-requisitos)
basic_skills = graph.get_basic_skills()
print(f"   • Habilidades básicas: {len(basic_skills)}")
print(f"     {', '.join(basic_skills)}")


CONSTRUÇÃO DO GRAFO DE HABILIDADES

✅ Grafo construído com sucesso!
   • Total de nós: 12
   • Total de arestas (pré-requisitos): 9
   • Habilidades básicas: 5
     S2, S1, H12, H10, S7


### 🔴 **VALIDAÇÃO CRÍTICA DO GRAFO (OBRIGATÓRIA)**

Esta etapa é **ESSENCIAL** e previne resultados incorretos em TODOS os desafios!

In [64]:
# ============================================================================
# VALIDAÇÃO CRÍTICA DO GRAFO
# ============================================================================

print("\n" + "=" * 70)
print("🔍 VALIDAÇÃO CRÍTICA DO GRAFO (OBRIGATÓRIA)")
print("=" * 70)

# Executa validação completa
validation_result = validate_graph(graph)

# Exibe relatório detalhado
print_validation_report(validation_result)

# Verifica se grafo é válido
if not validation_result['valid']:
    print("\n" + "=" * 70)
    print("❌ ERRO CRÍTICO: Grafo inválido!")
    print("=" * 70)
    print("\n🚫 NÃO é possível prosseguir com os desafios.")
    print("   Corrija os erros antes de continuar.")
    
    # Lança exceção para interromper execução
    raise ValueError(f"Grafo inválido: {validation_result['error_msg']}")
else:
    print("\n" + "=" * 70)
    print("✅ GRAFO VALIDADO COM SUCESSO!")
    print("=" * 70)


🔍 VALIDAÇÃO CRÍTICA DO GRAFO (OBRIGATÓRIA)

🔍 RELATÓRIO DE VALIDAÇÃO DO GRAFO

✅ STATUS: GRAFO VÁLIDO
   O grafo passou em todas as validações!

✅ CICLOS: Nenhum ciclo detectado (grafo é DAG)

✅ ÓRFÃOS: Todos os pré-requisitos existem

⚠️  AVISOS (1):
   • Aviso: 2 nó(s) isolado(s) (sem pré-requisitos e sem dependentes): ['H12', 'H10']

📊 ESTATÍSTICAS DO GRAFO:
   • Total de nós: 12
   • Total de arestas: 9
   • Habilidades básicas: 5

✅ GRAFO VALIDADO COM SUCESSO!


### 📊 **Resumo Detalhado do Grafo**

In [65]:
# ============================================================================
# RESUMO DO GRAFO
# ============================================================================

print_graph_summary(graph)

# Estatísticas adicionais
print("\n📊 Estatísticas Adicionais:")

# Calcula totais
total_time = sum(graph.get_metadata(s)['tempo_horas'] for s in graph.nodes)
total_value = sum(graph.get_metadata(s)['valor'] for s in graph.nodes)
total_complexity = sum(graph.get_metadata(s)['complexidade'] for s in graph.nodes)

print(f"   • Tempo total de todas as habilidades: {total_time}h")
print(f"   • Valor total de todas as habilidades: {total_value}")
print(f"   • Complexidade total de todas as habilidades: {total_complexity}")

# Graus de entrada/saída
print(f"\n📈 Distribuição de Pré-requisitos:")
for skill_id in graph.nodes:
    in_degree = graph.get_in_degree(skill_id)
    out_degree = graph.get_out_degree(skill_id)
    if in_degree > 0 or out_degree > 0:
        metadata = graph.get_metadata(skill_id)
        print(f"   • {skill_id} ({metadata['nome'][:30]}...): {in_degree} pré-req(s), {out_degree} dependente(s)")

RESUMO DO GRAFO DE HABILIDADES

📊 Estatísticas Gerais:
   • Total de habilidades: 12
   • Total de pré-requisitos: 9

🔵 Habilidades Básicas (sem pré-requisitos): 5
   • S2 - Modelagem de Dados (SQL)
   • S1 - Programação Básica (Python)
   • H12 - Introdução a IoT
   • H10 - Segurança de Dados
   • S7 - Estruturas em Nuvem (AWS/Azure)

📋 Ordenação Topológica:
   S2 → S1 → H12 → H10 → S7 → S5 → S3 → S8 → S4 → S9 → S6 → H11

📊 Estatísticas Adicionais:
   • Tempo total de todas as habilidades: 1000h
   • Valor total de todas as habilidades: 74
   • Complexidade total de todas as habilidades: 77

📈 Distribuição de Pré-requisitos:
   • S5 (Visualização de Dados (BI)...): 1 pré-req(s), 0 dependente(s)
   • H11 (Análise de Big Data...): 1 pré-req(s), 0 dependente(s)
   • S8 (APIs e Microsserviços...): 1 pré-req(s), 1 dependente(s)
   • S9 (DevOps & CI/CD...): 2 pré-req(s), 0 dependente(s)
   • S3 (Algoritmos Avançados...): 1 pré-req(s), 1 dependente(s)
   • S2 (Modelagem de Dados (SQL)...): 0

## 🎯 **Desafio 1 - Caminho de Valor Máximo**

### 📋 **Objetivo do Desafio 1**

Encontrar a sequência de habilidades que **maximiza o valor esperado** até alcançar **S6 (IA Generativa Ética)**, respeitando:
- ⏱️ Tempo total ≤ **350 horas**
- 🧠 Complexidade cumulativa ≤ **30**
- 🔗 Pré-requisitos obrigatórios

---

#### 🔧 **Abordagens**

**1. Solução Determinística (DP Knapsack Multidimensional)**
- **Algoritmo:** Programação Dinâmica
- **Dimensões:** Tempo (T) e Complexidade (C)
- **Recorrência:** 
```
  DP[i][t][c] = max(
      DP[i-1][t][c],                      # Não pega skill i
      DP[i-1][t-T[i]][c-C[i]] + V[i]     # Pega skill i
  )
```
- **Complexidade:** O(n × T × C) = O(12 × 350 × 30) ≈ O(126,000)

**2. Solução Estocástica (Monte Carlo)**
- **Simulação:** 1000 cenários
- **Incerteza:** V ~ Uniforme[V-10%, V+10%]
- **Objetivo:** Maximizar E[Valor total]
- **Métrica:** Desvio padrão e IC 95%

---

### 📊 **Execução Completa**

In [67]:
# ============================================================================
# DESAFIO 1 - EXECUÇÃO COMPLETA
# ============================================================================

# Executa solução completa (determinística + estocástica + comparação)
desafio1_results = solve_desafio1(
    graph=graph,
    max_time=MAX_TIME_HOURS,
    max_complexity=MAX_COMPLEXITY,
    target_skill=TARGET_SKILL,
    n_scenarios=1000,
    uncertainty=0.10,
    seed=GLOBAL_SEED
)


🔍 Executando: solve_complete
----------------------------------------------------------------------

🎯 DESAFIO 1 - CAMINHO DE VALOR MÁXIMO

📊 FASE 1: Solução Determinística (sem incerteza)
----------------------------------------------------------------------

🔍 Executando: solve_deterministic
----------------------------------------------------------------------

📊 Performance:
   ⏱️  Tempo: 1.08 s
   💾 Memória: 10407.80 KB

✅ Solução determinística:
   Caminho: H12 → H10 → S5 → S9 → H11
   Valor: 31.00
   Tempo: 330h / 350h
   Complexidade: 30 / 30
   Alcança S6: ❌ NÃO

📊 FASE 2: Solução Estocástica (Monte Carlo - 1000 cenários)
----------------------------------------------------------------------

🔍 Executando: solve_stochastic
----------------------------------------------------------------------
Gerando 1000 cenários com incerteza ±10.0%...
Executando simulação Monte Carlo...


KeyboardInterrupt: 

### 📈 **Análise dos Resultados**

In [ ]:
# ============================================================================
# ANÁLISE DETALHADA DOS RESULTADOS
# ============================================================================

print("\n" + "=" * 70)
print("📊 ANÁLISE DETALHADA - DESAFIO 1")
print("=" * 70)

# Extrai resultados
det_results = desafio1_results['deterministic']
sto_results = desafio1_results['stochastic']
comparison = desafio1_results['comparison']

# Solução determinística
print("\n1️⃣ SOLUÇÃO DETERMINÍSTICA:")
print("-" * 70)
det_solution = det_results['solution']
print(f"   Caminho: {' → '.join(det_solution['path'])}")
print(f"   Valor total: {det_solution['total_value']:.2f}")
print(f"   Tempo total: {det_solution['total_time']}h / {MAX_TIME_HOURS}h ({det_solution['total_time']/MAX_TIME_HOURS*100:.1f}%)")
print(f"   Complexidade: {det_solution['total_complexity']} / {MAX_COMPLEXITY} ({det_solution['total_complexity']/MAX_COMPLEXITY*100:.1f}%)")
print(f"   Alcança S6: {'✅ SIM' if det_solution['reaches_target'] else '❌ NÃO'}")

# Solução estocástica
print("\n2️⃣ SOLUÇÃO ESTOCÁSTICA (MONTE CARLO):")
print("-" * 70)
mc_result = sto_results['monte_carlo_result']
print(f"   E[Valor] = {mc_result.expected_value:.2f} ± {mc_result.std_deviation:.2f}")
print(f"   Mediana: {mc_result.median:.2f}")
print(f"   Range: [{mc_result.min_value:.2f}, {mc_result.max_value:.2f}]")
print(f"   IC 95%: [{mc_result.confidence_interval_95[0]:.2f}, {mc_result.confidence_interval_95[1]:.2f}]")
print(f"   Coef. Variação: {(mc_result.std_deviation/mc_result.expected_value*100):.2f}%")

# Comparação
print("\n3️⃣ COMPARAÇÃO DETERMINÍSTICO vs ESTOCÁSTICO:")
print("-" * 70)
print(f"   Determinístico: {comparison['deterministic_value']:.2f}")
print(f"   Estocástico: {comparison['stochastic_expected']:.2f} ± {comparison['stochastic_std']:.2f}")
print(f"   Diferença absoluta: {comparison['difference']:.2f}")
print(f"   Erro relativo: {comparison['relative_error_percent']:.2f}%")
print(f"   Det. dentro IC 95%: {'✅ SIM' if comparison['deterministic_within_95ci'] else '❌ NÃO'}")

print(f"\n💡 Interpretação:")
print(f"   {comparison['interpretation']}")

### 📝 **Detalhes do Caminho Ótimo**

In [ ]:
# ============================================================================
# DETALHES DO CAMINHO SELECIONADO
# ============================================================================

print("\n" + "=" * 70)
print("📝 DETALHAMENTO DO CAMINHO ÓTIMO (DETERMINÍSTICO)")
print("=" * 70)

print(f"\n🎯 Sequência: {' → '.join(det_solution['path'])}")

print(f"\n📊 Habilidades selecionadas:")
for i, skill_detail in enumerate(det_solution['skill_details'], 1):
    print(f"\n   {i}. {skill_detail['skill_id']} - {skill_detail['nome']}")
    print(f"      • Tempo: {skill_detail['tempo']}h")
    print(f"      • Valor: {skill_detail['valor']}")
    print(f"      • Complexidade: {skill_detail['complexidade']}")
    
    # Pré-requisitos
    prereqs = graph.get_prerequisites(skill_detail['skill_id'])
    if prereqs:
        print(f"      • Pré-requisitos: {', '.join(prereqs)}")
    else:
        print(f"      • Pré-requisitos: Nenhum (habilidade básica)")

print(f"\n✅ Caminho válido e otimizado!")

### 📊 **Visualizações - Desafio 1**

In [ ]:
# ============================================================================
# VISUALIZAÇÕES - DESAFIO 1
# ============================================================================

print("\n📊 Gerando visualizações do Desafio 1...")

# Histograma Monte Carlo
fig1 = plot_monte_carlo_distribution(
    values=mc_result.all_results,
    deterministic_value=det_solution['total_value'],
    title="Desafio 1: Distribuição Monte Carlo - Valor Total",
    save_path=OUTPUTS_DIR / 'figures' / 'desafio1_monte_carlo.png'
)
plt.show()

print(f"✅ Gráfico salvo em: {OUTPUTS_DIR / 'figures' / 'desafio1_monte_carlo.png'}")

In [ ]:
# ============================================================================
# SALVAMENTO DOS RESULTADOS
# ============================================================================

# Salva resultados em JSON
save_desafio1_results(det_results, sto_results, comparison)

print("\n✅ Desafio 1 concluído com sucesso!")